In [ ]:
#pip install -q -U google-genai

# Importacion de librerias

In [1]:
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

# Librerias basicas
import os, tempfile, glob, random, json
from pathlib import Path
import numpy as np
from itertools import combinations
from uuid import uuid4
from PIL import Image

from IPython.display import Markdown
from getpass import getpass

# Para el entorno de google
from google import genai
from google.genai import types

# Langchain 
from langchain_google_genai import ChatGoogleGenerativeAI, GoogleGenerativeAIEmbeddings

from langchain_core.prompts import PromptTemplate, ChatPromptTemplate
from langchain_core.runnables import RunnableLambda, RunnableParallel, RunnablePassthrough, RunnableSequence
from langchain_core.messages import AIMessage, HumanMessage, get_buffer_string, ChatMessage
from langchain_core.documents import Document
from langchain_core.output_parsers import StrOutputParser
from langchain_community.document_transformers import EmbeddingsRedundantFilter,LongContextReorder
from langchain_community.chat_message_histories import StreamlitChatMessageHistory
from operator import itemgetter
from langchain_core.prompts.base import format_document


# Document Loaders
from langchain_community.document_loaders import (
    PyPDFLoader,
    TextLoader,
    DirectoryLoader,
    CSVLoader,
    UnstructuredExcelLoader,
    Docx2txtLoader,
    JSONLoader
)

# Text Splitter
from langchain_text_splitters import RecursiveCharacterTextSplitter, CharacterTextSplitter

# Chroma: vectorstore
from langchain_chroma import Chroma


## Api key de google gemini

In [27]:
os.environ["GOOGLE_API_KEY"] = "api_key"

In [ ]:
#client = genai.Client(api_key="api_key")

In [ ]:
from google import genai

client = genai.Client()

result = client.models.embed_content(
        model="gemini-embedding-001",
        contents="What is the meaning of life?")

print(result.embeddings)

# Cargado de datos

## Cargado del JSON

In [3]:
with open("../resources/cpe.json", "r", encoding="utf-8") as f:
    data = json.load(f)

docs = []

In [4]:
for item in data:
    tipo = item.get("tipo", "").capitalize()
    text = ""

    if tipo == "Introduccion":
        text = f"Título: {item.get('titulo', '')}\n" \
                f"Subtítulo: {item.get('subtitulo', '')}\n" \
                f"{item.get('contenido', '')}"

    elif tipo == "Articulo":
        text = (
            f"Parte {item.get('parte_num', '')}: {item.get('parte_nom', '')}\n"
            f"Título {item.get('titulo_num', '')}: {item.get('titulo_nom', '')}\n"
            f"Capítulo {item.get('capitulo_num', '')}: {item.get('capitulo_nom', '')}\n"
            f"Sección {item.get('seccion_num', '')}: {item.get('seccion_nom', '')}\n"
            f"Artículo {item.get('art_num', '')}: {item.get('nombre_juridico', '')}\n"
            f"{item.get('contenido', '')}"
        )
        
    elif tipo == "Disposición":
        text = (
            f"Disposición {item.get('disposicion', '')}\n"
            f"Nombre jurídico: {item.get('nombre_juridico', '')}\n"
            f"{item.get('contenido', '')}"
        )

    # Pongo este else, porque sino me sale error en page_content=text, porque es posible que text este vacio
    else:
        text = item.get("contenido", "")

    docs.append(Document(page_content=text, metadata={"tipo": tipo}))

print(f"Se cargaron {len(docs)} documentos desde el JSON.")

Se cargaron 426 documentos desde el JSON.


In [5]:
# Solamente para testear que se hayan cargado los documentos correctamente
import random
random_document_id = random.choice(range(len(docs)))

print("test: ", random_document_id)
print(docs[random_document_id])

test:  261
page_content='Parte Segunda: Estructura y organización funcional del estado
Título 8: Relaciones internacionales, fronteras, integración y reivindicación marítima
Capítulo Primero: Relaciones Internacionales
Sección None: None
Artículo 260: Denuncia de los Tratados Internacionales
I. La denuncia de los tratados internacionales seguirá los procedimientos establecidos en el propio tratado internacional, las normas generales del Derecho internacional, y los procedimientos establecidos en la Constitución y la ley para su ratificación. II. La denuncia de los tratados ratificados deberá ser aprobada por la Asamblea Legislativa Plurinacional antes de ser ejecutada por la Presidenta o Presidente del Estado. III. Los tratados aprobados por referendo deberán ser sometidos a un nuevo referendo antes de su denuncia por la Presidenta o Presidente del Estado.' metadata={'tipo': 'Articulo'}


## Embedding part

In [6]:
model_embedding = GoogleGenerativeAIEmbeddings(
    model="models/embedding-001",
    task_type="retrieval_document"
)

## Create Vector Store

In [7]:
def create_vectorstore(collection_name, model_embedding, vectorstore_name):
    persist_directory = ("./" + vectorstore_name)
    vector_store = Chroma(
        collection_name = collection_name,
        embedding_function=model_embedding,
        persist_directory = persist_directory
    )
    return vector_store, vectorstore_name

In [8]:
create_new_vectorstore = True # Cambiar a true si queremos volver a crear la bd vectorial
if create_new_vectorstore:
    vector_store_gemini,_ = create_vectorstore(
        collection_name = "cpep_gemini",
        model_embedding = model_embedding,
        vectorstore_name= "vectordb/cpep_gemini"
    )
    print("vector_store_gemini:",vector_store_gemini._collection.count(),"documentos.")

vector_store_gemini: 0 documentos.


## Asignamiento de IDs

In [9]:
ids = []
metas = []
cont = 0

for i in range(len(docs)):
    if (i<2):
        ids.append(f"introduccion_{i+1}")
        metas.append({"tipo": "introduccion", "número": i+1})
    elif(i<399):
        ids.append(f"artículo_{i-1}")
        metas.append({"tipo": "artículo", "número": i-1})
    elif(i==399):
        ids.append(f"artículo_398_A")
        metas.append({"tipo": "artículo", "número": "398_A"})
    elif(i==400):
        ids.append(f"artículo_398_B")
        metas.append({"tipo": "artículo", "número": "398_B"})
    elif(i<414):
        ids.append(f"artículo_{i-2}")
        metas.append({"tipo": "artículo", "número": i-2})
    else:
        cont += 1
        ids.append(f"disposición_{cont}")
        metas.append({"tipo": "disposicion", "número": cont})

assert len(docs) == len(ids) == len(metas)

In [10]:
import numpy as np
lengths = [len(d.page_content) for d in docs]
print("Promedio:", np.mean(lengths), " | Máximo:", max(lengths))

Promedio: 866.018779342723  | Máximo: 5162


## Add to vector store

In [11]:
vector_store_gemini.add_documents(documents=docs, ids=ids)
print("vector_store_gemini:", vector_store_gemini._collection.count(), "documentos.")

vector_store_gemini: 426 documentos.


In [12]:
print(vector_store_gemini._collection.count())
print(vector_store_gemini._collection.peek())

426
{'ids': ['introduccion_1', 'introduccion_2', 'artículo_1', 'artículo_2', 'artículo_3', 'artículo_4', 'artículo_5', 'artículo_6', 'artículo_7', 'artículo_8'], 'embeddings': array([[ 0.03481974,  0.02267552, -0.05910448, ...,  0.07081398,
        -0.00729247, -0.01907829],
       [ 0.02480937,  0.00854049, -0.05155088, ...,  0.06905004,
         0.01012001, -0.03217785],
       [ 0.03369951, -0.02515825, -0.06354076, ...,  0.08418227,
         0.00432602, -0.02403576],
       ...,
       [ 0.05015473, -0.01955299, -0.04484588, ...,  0.07034896,
         0.01142905, -0.00240585],
       [ 0.02327036,  0.00135682, -0.02365066, ...,  0.08782235,
        -0.01484509, -0.00636925],
       [ 0.02374903,  0.01135887, -0.04797747, ...,  0.05730618,
        -0.01343114, -0.02842696]], shape=(10, 768)), 'documents': ['Título: Antecedentes Legales\nSubtítulo: None\nLa Constitución Política del Estado, promulgada el 7 de febrero de 2009, consta de 5 Partes: I. BASES FUNDAMENTALES DEL ESTADO II. 

## Call the vector store

In [13]:
persist_directory = "vectordb/cpep_gemini"
collection_name = "cpep_gemini"

In [14]:
vector_store_gemini = Chroma(
    collection_name=collection_name,
    embedding_function=model_embedding,
    persist_directory=persist_directory
)

In [16]:
print("vector_store_gemini:", vector_store_gemini._collection.count(), "documentos.")
print(vector_store_gemini._collection.count())
print(vector_store_gemini._collection.peek())

vector_store_gemini: 426 documentos.
426
{'ids': ['introduccion_1', 'introduccion_2', 'artículo_1', 'artículo_2', 'artículo_3', 'artículo_4', 'artículo_5', 'artículo_6', 'artículo_7', 'artículo_8'], 'embeddings': array([[ 0.03481974,  0.02267552, -0.05910448, ...,  0.07081398,
        -0.00729247, -0.01907829],
       [ 0.02480937,  0.00854049, -0.05155088, ...,  0.06905004,
         0.01012001, -0.03217785],
       [ 0.03369951, -0.02515825, -0.06354076, ...,  0.08418227,
         0.00432602, -0.02403576],
       ...,
       [ 0.05015473, -0.01955299, -0.04484588, ...,  0.07034896,
         0.01142905, -0.00240585],
       [ 0.02327036,  0.00135682, -0.02365066, ...,  0.08782235,
        -0.01484509, -0.00636925],
       [ 0.02374903,  0.01135887, -0.04797747, ...,  0.05730618,
        -0.01343114, -0.02842696]], shape=(10, 768)), 'documents': ['Título: Antecedentes Legales\nSubtítulo: None\nLa Constitución Política del Estado, promulgada el 7 de febrero de 2009, consta de 5 Partes: I

## Similarity Search

In [17]:
def print_documents(docs,search_with_score=False):
    if search_with_score:
        print(
            f"\n{'-' * 100}\n".join(
                [f"Document {i+1}:\n\n" + doc[0].page_content +"\n\nscore:"+str(round(doc[-1],3))+"\n" 
                for i, doc in enumerate(docs)]
            )
        )
    else:
        # used for similarity_search or max_marginal_relevance_search
        print(
            f"\n{'-' * 100}\n".join(
                [f"Document {i+1}:\n\n" + doc.page_content 
                for i, doc in enumerate(docs)]
            )
        )  

In [24]:
query = '¿Qué dice el artículo 320 de la constitución política del estado Boliviano?'
docs_withScores = vector_store_gemini.similarity_search_with_score(query,k=5)

print_documents(docs_withScores,search_with_score=True)

Document 1:

Disposición Abrogatoria
Nombre jurídico: None
Queda abrogada la Constitución Política del Estado de 1967 y sus reformas posteriores.

score:0.329

----------------------------------------------------------------------------------------------------
Document 2:

Disposición Final
Nombre jurídico: None
Esta Constitución, aprobada en referendo por el pueblo boliviano entrará en vigencia el día de su publicación en la Gaceta Oficial.

score:0.348

----------------------------------------------------------------------------------------------------
Document 3:

Parte Cuarta: Estructura y organización económica del Estado
Título 2: Medio Ambiente, Recursos Naturales, Tierra y Territorio
Capítulo Primero: Medio Ambiente
Sección None: None
Artículo 344: Prohibición a la Fabricación y Uso de Armas Químicas, Biológicas y Nucleares
. Se prohíbe la fabricación y uso de armas químicas, biológicas y nucleares en el territorio boliviano, así como la internación, tránsito y depósito de resi

In [25]:
query_embeddings = model_embedding.embed_query(query)
docs_embeddings = model_embedding.embed_documents(
    [docs_withScores[i][0].page_content 
    for i in range(len(docs_withScores))
    ]
)

for i in range(len(docs_embeddings)):
    dot_product = round(np.dot(query_embeddings, docs_embeddings[i]),4)
    print(f"Similarty of document_{i} to the query: {dot_product}")

Similarty of document_0 to the query: 0.8357
Similarty of document_1 to the query: 0.8259
Similarty of document_2 to the query: 0.8162
Similarty of document_3 to the query: 0.8139
Similarty of document_4 to the query: 0.8133


In [26]:
print(docs[321])

page_content='Parte Cuarta: Estructura y organización económica del Estado
Título 1: Organización económica del Estado
Capítulo Tercero: Políticas económicas
Sección None: None
Artículo 320: Priorización de la Inversión Boliviana e Inversión Extranjera
I. La inversión boliviana se priorizará frente a la inversión extranjera. II. Toda inversión extranjera estará sometida a la jurisdicción, a las leyes y a las autoridades bolivianas, y nadie podrá invocar situación de excepción, ni apelar a reclamaciones diplomáticas para obtener un tratamiento más favorable. III. Las relaciones económicas con estados o empresas extranjeras se realizarán en condiciones de independencia, respeto mutuo y equidad. No se podrá otorgar a Estados o empresas extranjeras condiciones más beneficiosas que las establecidas para los bolivianos. IV. El Estado es independiente en todas las decisiones de política económica interna, y no aceptará imposiciones ni condicionamientos sobre esta política por parte de estados

# Retrievers

## Vector Store-backed retriever